In [18]:
#python libraries for handle data manipulation, machine leaning, and plotting.
!pip install tensorflow pandas numpy matplotlib scikit-learn


In [19]:
!pip install opendatasets -q

import opendatasets as od

# download dataset from kaggle.
od.download("https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset")


Skipping, found downloaded files in "./movielens-20m-dataset" (use force=True to force download)


In [20]:
import pandas as pd

# Load the ratings and movies data
ratings = pd.read_csv('./movielens-20m-dataset/rating.csv')  #contains information about movie ratings from users including the userId,movieId and ratings
movies = pd.read_csv('./movielens-20m-dataset/movie.csv')    #contains information abput movies,including the movieId and the movie title

# Merge the datasets and provides a combined data set
data = pd.merge(ratings, movies, on='movieId')

# Inspect the merged data
print(data.head())


   userId  movieId  rating            timestamp  \
0       1        2     3.5  2005-04-02 23:53:47   
1       1       29     3.5  2005-04-02 23:31:16   
2       1       32     3.5  2005-04-02 23:33:39   
3       1       47     3.5  2005-04-02 23:32:07   
4       1       50     3.5  2005-04-02 23:29:40   

                                               title  \
0                                     Jumanji (1995)   
1  City of Lost Children, The (Cité des enfants p...   
2          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
3                        Seven (a.k.a. Se7en) (1995)   
4                         Usual Suspects, The (1995)   

                                   genres  
0              Adventure|Children|Fantasy  
1  Adventure|Drama|Fantasy|Mystery|Sci-Fi  
2                 Mystery|Sci-Fi|Thriller  
3                        Mystery|Thriller  
4                  Crime|Mystery|Thriller  


In [21]:
# Merge ratings with movies on the 'movieId' column
data = pd.merge(ratings, movies, on='movieId')

# Inspect merged data
print(data.head())


   userId  movieId  rating            timestamp  \
0       1        2     3.5  2005-04-02 23:53:47   
1       1       29     3.5  2005-04-02 23:31:16   
2       1       32     3.5  2005-04-02 23:33:39   
3       1       47     3.5  2005-04-02 23:32:07   
4       1       50     3.5  2005-04-02 23:29:40   

                                               title  \
0                                     Jumanji (1995)   
1  City of Lost Children, The (Cité des enfants p...   
2          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
3                        Seven (a.k.a. Se7en) (1995)   
4                         Usual Suspects, The (1995)   

                                   genres  
0              Adventure|Children|Fantasy  
1  Adventure|Drama|Fantasy|Mystery|Sci-Fi  
2                 Mystery|Sci-Fi|Thriller  
3                        Mystery|Thriller  
4                  Crime|Mystery|Thriller  


In [22]:
# Data cleaning
# Check for missing values
print(data.isnull().sum())

# Drop rows with missing values
data = data.dropna()

# Inspect the cleaned data
print(data.info())


userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
 4   title      object 
 5   genres     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB
None


In [23]:
#data exploration
print("Dataset size:", data.memory_usage(deep=True).sum() / 1e6, "MB") #display the size of the dataset in MB
print("Number of unique users:", data['userId'].nunique()) #identify the number of unique users in the dataset
print("Number of unique movies:", data['title'].nunique()) #identify teh numbe of unique movies in the dataset


Dataset size: 5160.339787 MB
Number of unique users: 138493
Number of unique movies: 26729


In [24]:
# Reduce dataset to the top 1000 users and top 1000 movies as it is a large dataset
top_users = data['userId'].value_counts().head(1000).index
top_movies = data['title'].value_counts().head(1000).index

reduced_data = data[(data['userId'].isin(top_users)) & (data['title'].isin(top_movies))]

# Create the user-movie matrix-->rows represnt users and columns represent movies and values represent ratings
#missing values ae filled with 0-->means that no rating for a particular movie by that user
user_movie_matrix = reduced_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)

print("User-Movie Matrix Shape:", user_movie_matrix.shape)


User-Movie Matrix Shape: (1000, 1000)


In [25]:

# Import required libraries
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Calculate Cosine Similarity between movies based on the user ratings
movie_similarity = cosine_similarity(user_movie_matrix.T)
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index=user_movie_matrix.columns,
    columns=user_movie_matrix.columns,
)
# this results in a similarity matrix where each entry indicate how similar two movies are based on their ratings
print("Movie Similarity Matrix Shape:", movie_similarity_df.shape)

# Step 2: Recommend Movies Based on a Single Movie
#this function takes the movie title and returns the top N most similar movies based on the similar matrix
def recommend_movies(movie_title, similarity_matrix, top_n=5):
    if movie_title not in similarity_matrix.index:
        return [f"Movie '{movie_title}' not found in the dataset."]

    # Get the similarity scores for the given movie
    similar_movies = similarity_matrix[movie_title].sort_values(ascending=False).iloc[1:top_n+1]
    return similar_movies.index.tolist()

# Example: Recommend movies based on a specific title
#this function recommends movies based on the movies they have rated highly. It combines recommendations from all the movies the user has rated and sorts them by similarity.
movie_title = "Toy Story (1995)"  # Replace with a title in your dataset
recommended_movies = recommend_movies(movie_title, movie_similarity_df)

print(f"Movies similar to '{movie_title}':", recommended_movies)

# Step 3: Create a Recommendation Function for Users
def recommend_for_user(user_id, user_movie_matrix, similarity_matrix, top_n=5):
    if user_id not in user_movie_matrix.index:
        return [f"User ID '{user_id}' not found in the dataset."]

    # Get user's movie ratings
    user_ratings = user_movie_matrix.loc[user_id]
    rated_movies = user_ratings[user_ratings > 0].index

    if len(rated_movies) == 0:
        return ["This user has not rated any movies yet."]

    # Get recommendations for all rated movies and combine
    recommendations = {}
    for movie in rated_movies:
        similar_movies = recommend_movies(movie, similarity_matrix, top_n=top_n)
        for sm in similar_movies:
            if sm not in rated_movies:  # Exclude already rated movies
                recommendations[sm] = recommendations.get(sm, 0) + 1

    # Sort recommendations by score
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return [movie for movie, score in sorted_recommendations[:top_n]]

# Recommend movies for a user
user_id = 1  # Replace with a user ID in your dataset
user_recommendations = recommend_for_user(user_id, user_movie_matrix, movie_similarity_df)

print(f"Movies recommended for user {user_id}:", user_recommendations)


Movie Similarity Matrix Shape: (1000, 1000)
Movies similar to 'Toy Story (1995)': ['Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Back to the Future (1985)', 'Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)', 'Matrix, The (1999)']
Movies recommended for user 1: ["User ID '1' not found in the dataset."]


In [26]:
# evaluation metric: Coverage
rated_movies = data['title'].nunique()
recommended_movies = len(set(user_recommendations))
coverage = (recommended_movies / rated_movies) * 100
print(f"Coverage: {coverage:.2f}%")




Coverage: 0.00%


In [27]:

import pickle

# Save the similarity matrix
with open('movie_similarity.pkl', 'wb') as file:
    pickle.dump(movie_similarity_df, file)

# Save the user-movie matrix
with open('user_movie_matrix.pkl', 'wb') as file:
    pickle.dump(user_movie_matrix, file)
    # allows for reloading the models without recalculating them.


In [28]:
#Reloads the previously saved models for use without recomputing them.
with open('movie_similarity.pkl', 'rb') as file:
    movie_similarity_df = pickle.load(file)

with open('user_movie_matrix.pkl', 'rb') as file:
    user_movie_matrix = pickle.load(file)


In [29]:
# Check movie similarity matrix
print(movie_similarity_df.head())

# Check user-movie matrix
print(user_movie_matrix.head())


title                                               (500) Days of Summer (2009)  \
title                                                                             
(500) Days of Summer (2009)                                            1.000000   
10 Things I Hate About You (1999)                                      0.492952   
101 Dalmatians (1996)                                                  0.373397   
101 Dalmatians (One Hundred and One Dalmatians)...                     0.381708   
12 Angry Men (1957)                                                    0.497810   

title                                               10 Things I Hate About You (1999)  \
title                                                                                   
(500) Days of Summer (2009)                                                  0.492952   
10 Things I Hate About You (1999)                                            1.000000   
101 Dalmatians (1996)                                         

In [30]:

# Recommend movies for a specific movie
movie_title = "Toy Story (1995)"  # Replace with an existing title
recommended_movies = recommend_movies(movie_title, movie_similarity_df)
print(f"Movies similar to '{movie_title}':", recommended_movies)

# Recommend movies for a specific user
user_id = 1  # Replace with a valid user ID
user_recommendations = recommend_for_user(user_id, user_movie_matrix, movie_similarity_df)
print(f"Movies recommended for user {user_id}:", user_recommendations)


Movies similar to 'Toy Story (1995)': ['Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Back to the Future (1985)', 'Jurassic Park (1993)', 'Star Wars: Episode IV - A New Hope (1977)', 'Matrix, The (1999)']
Movies recommended for user 1: ["User ID '1' not found in the dataset."]


In [32]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dropdown for selecting functionality
functionality_dropdown = widgets.Dropdown(
    options=[
        'Find Similar Movies',
        'Recommend Movies for User',
        'Dataset Summary',
        'Top Users and Movies'
    ],
    description='Function:',
    layout=widgets.Layout(width='50%')
)

# Movie dropdown for similarity
movie_dropdown = widgets.Dropdown(
    options=user_movie_matrix.columns,
    description='Select Movie:',
    layout=widgets.Layout(width='50%'),
    visible=False
)

# User dropdown for user-specific recommendations
user_dropdown = widgets.Dropdown(
    options=user_movie_matrix.index,
    description='Select User:',
    layout=widgets.Layout(width='50%'),
    visible=False
)

# Button for executing the selected functionality
execute_button = widgets.Button(description="Execute", button_style='success')

# Output area for displaying results
output_area = widgets.Output()

# Function to handle execution based on the selected functionality
def execute_functionality(b):
    with output_area:
        clear_output()
        selected_function = functionality_dropdown.value

        if selected_function == 'Find Similar Movies':
            movie_title = movie_dropdown.value
            recommendations = recommend_movies(movie_title, movie_similarity_df)
            print(f"Movies similar to '{movie_title}':")
            for movie in recommendations:
                print(f"- {movie}")

        elif selected_function == 'Recommend Movies for User':
            user_id = user_dropdown.value
            user_recommendations = recommend_for_user(user_id, user_movie_matrix, movie_similarity_df)
            print(f"Movies recommended for user {user_id}:")
            for movie in user_recommendations:
                print(f"- {movie}")

        elif selected_function == 'Dataset Summary':
            print("Dataset Summary:")
            print(f"- Number of unique users: {data['userId'].nunique()}")
            print(f"- Number of unique movies: {data['title'].nunique()}")
            print(f"- User-Movie Matrix shape: {user_movie_matrix.shape}")
            print(f"- Dataset size: {data.memory_usage(deep=True).sum() / 1e6:.2f} MB")

        elif selected_function == 'Top Users and Movies':
            print("Top 5 Users (by number of ratings):")
            print(data['userId'].value_counts().head(5))
            print("\nTop 5 Movies (by number of ratings):")
            print(data['title'].value_counts().head(5))

# Bind the button to the execution function
execute_button.on_click(execute_functionality)

# Function to update visibility of dropdowns based on selected functionality
def update_dropdown_visibility(change):
    if change.new == 'Find Similar Movies':
        movie_dropdown.layout.display = 'block'
        user_dropdown.layout.display = 'none'
    elif change.new == 'Recommend Movies for User':
        movie_dropdown.layout.display = 'none'
        user_dropdown.layout.display = 'block'
    else:
        movie_dropdown.layout.display = 'none'
        user_dropdown.layout.display = 'none'

# Observe changes in the functionality dropdown
functionality_dropdown.observe(update_dropdown_visibility, names='value')

# Display the widgets
display(functionality_dropdown, movie_dropdown, user_dropdown, execute_button, output_area)



Dropdown(description='Function:', layout=Layout(width='50%'), options=('Find Similar Movies', 'Recommend Movie…

Dropdown(description='Select Movie:', layout=Layout(width='50%'), options=('(500) Days of Summer (2009)', '10 …

Dropdown(description='Select User:', layout=Layout(width='50%'), options=(156, 208, 359, 572, 586, 741, 768, 7…

Button(button_style='success', description='Execute', style=ButtonStyle())

Output()